In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math 

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
@pyimport numpy
@pyimport PIL
@pyimport skimage.transform as skimage_transform
@pyimport sklearn.cluster as cluster
@pyimport matplotlib.cm as cm

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")

In [ ]:
corner_cue = 
[
    ["20220406_111526", 9, "20220406_125842", 9, "jen"],
    ["20220407_090156", 5, "20220407_104712", 5, "jen"],
    ["20220406_153842", 9, "20220406_171558", 9, "jen"]
]

In [ ]:
data_all_exp = reduce(vcat, [corner_cue])

In [ ]:
data_all = []
server_all = []
for data in data_all_exp
    if length(data).== 4
        append!(data_all, [data[1]])
        append!(data_all, [data[3]])
        append!(server_all, data[2])
        append!(server_all, data[4])
    elseif length(data).== 2
        append!(data_all, [data[1]])
        append!(server_all, data[2])
    end
end

In [ ]:
which_combo = 2
experimenter = "jen"

In [ ]:
experiment_filename_1 = data_all_exp[which_combo][1]
server = data_all_exp[which_combo][2]
ds_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_1 = Dataset(experiment_filename_1, "jen", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")

experiment_filename_2 = data_all_exp[which_combo][3]
server = data_all_exp[which_combo][4]
ds_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_2 = Dataset(experiment_filename_2, "jen", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")
ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")

In [ ]:
@pyimport matplotlib.patches as patches

In [ ]:
n_pos = 60

# orientation-corrected fish location (time binned)
position_file = h5open(joinpath(data_path(ds_save_cy_1), "oblique_corrected", "for_place_calculation_chamber_geometry_$(experiment_filename_1)_n60_oblique_corrected.h5"))
x_fish_sweep_mean = read(position_file,"x_fish_sweep_mean")
y_fish_sweep_mean = read(position_file,"y_fish_sweep_mean")
mask_valid = read(position_file,"mask_valid")
x_digital = read(position_file,"x_digital")
y_digital = read(position_file,"y_digital")
x_bins = read(position_file,"x_bins")
y_bins = read(position_file,"y_bins")
valid_moving_indices = read(position_file,"valid_moving_indices")
what_angle = read(position_file,"what_angle")
mask_valid = read(position_file,"mask_valid")
chamber_roi = read(position_file,"chamber_roi")
close(position_file)
mask_invalid = .!mask_valid;

In [ ]:
C, y_fish_ob, x_fish_ob, img_bg, heading_ob = h5open(ds_1, "behavior.h5"; raw = true) do file
    read(file, "C"), 
    read(file, "fish_yolk_y"), 
    read(file, "fish_yolk_x"),
    read(file, "img_bg"),
    read(file, "heading")
end;


global w = size(img_bg, 1)
global l = size(img_bg, 2)

In [ ]:
img_bg_end_1 = img_bg[:,:,end];

In [ ]:
NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
NMF_file = h5open(NMF_filename, "r")
global Z_all = HDF5.readmmap(NMF_file["Z_all"])
global X_all = HDF5.readmmap(NMF_file["X_all"])
global Y_all = HDF5.readmmap(NMF_file["Y_all"])
global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
global A_dFF_1 = HDF5.readmmap(NMF_file["A_dFF"]); # bs update
global A_baseline_1 = HDF5.readmmap(NMF_file["A_baseline"]); # bs update
close(NMF_file)


n_neuron = length(X_all);

In [ ]:
# match the frames if we have offsets
n_neurons = size(A_dFF_1, 2)
n_sweeps = size(A_dFF_1, 1)

x_fish_sweep_mean = x_fish_sweep_mean[1:n_sweeps]
y_fish_sweep_mean = y_fish_sweep_mean[1:n_sweeps];

In [ ]:
plot(x_fish_sweep_mean)

In [ ]:
file_folder_1 = joinpath(data_path(ds_save_cy_1), "oblique_corrected")
all_files = readdir(file_folder_1)
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = [occursin("A_dF", long_name_files[i])*occursin("neuron", long_name_files[i]) for i in 1:length(long_name_files)]
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
println(candidate_filename[which_file])
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
info_filename = joinpath(file_folder_1, save_file_name)
file = h5open(info_filename, "r")
place_map_all_1 = HDF5.readmmap(file["place_map_all"])
specificity_1 = HDF5.readmmap(file["specificity"])
specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
plot_index = HDF5.readmmap(file["bool_index"])
close(file)

In [ ]:
# real_cell_index = 84981
plot_index = copy(valid_moving_indices)
plot(plot_index);

In [ ]:
mask_valid = isfinite.(place_map_all_1[:,:,1])

mask_valid_index = findall(mask_valid)
mask_valid_x = [x[1] for x in mask_valid_index]
mask_valid_y = [x[2] for x in mask_valid_index];
mask_valid_x_min = minimum(mask_valid_x)
mask_valid_x_max = maximum(mask_valid_x)
mask_valid_y_min = minimum(mask_valid_y)
mask_valid_y_max = maximum(mask_valid_y);

In [ ]:
chamber_roi_mask = Float32.(copy(chamber_roi))
chamber_roi_mask[chamber_roi_mask.!=0].= NaN32;


img_bg_mask = copy(img_bg_end_1)
img_bg_mask[img_bg_mask.==0].=NaN32
img_bg_mask[Float32.(chamber_roi).!=0].=NaN32;

In [ ]:
imshow(chamber_roi_mask)

In [ ]:
imshow(chamber_roi)

chamber_roi[chamber_roi.!=0].=1;
countour = find_contours(chamber_roi)[1];
countour_array = hcat(getindex.(countour, 1), getindex.(countour,2));

In [ ]:
x_fish = zeros(Float32, length(x_fish_ob))
y_fish = zeros(Float32, length(y_fish_ob))
for i in 1:length(x_fish_ob)
x_fish[i], y_fish[i] = rotate2d(x_fish_ob[i], y_fish_ob[i], w/2, l/2, pi*what_angle/180)
end

heading = heading_ob.+pi*what_angle/180;

x_digital = numpy.digitize(x_fish, x_bins)
y_digital = numpy.digitize(y_fish, y_bins);

scatter(x_digital, y_digital)

In [ ]:
real_cell_index = 9097
neural_activity = A_dFF_1[:, real_cell_index];
neural_activity = (neural_activity.-numpy.nanstd(neural_activity))./numpy.nanstd(neural_activity);
corresponding_map = place_map_all_1[:,:,real_cell_index]
figure()
imshow(corresponding_map)
map_wo_nan = corresponding_map[.~isnan.(corresponding_map)]
where_max = findall(corresponding_map .== numpy.nanmax(map_wo_nan))
where_max_x = where_max[1][1]
where_max_y = where_max[1][2]
corresponding_xmin = x_bins[where_max_x-1]
corresponding_xmax = x_bins[where_max_x+1+1]
corresponding_ymin = y_bins[where_max_y-1]
corresponding_ymax = y_bins[where_max_y+1+1]
when_visit = intersect(findall(x_fish_sweep_mean.>=corresponding_xmin), findall(x_fish_sweep_mean.<=corresponding_xmax), findall(y_fish_sweep_mean.>=corresponding_ymin), findall(y_fish_sweep_mean.<=corresponding_ymax));
interval = 40 
when_visit_real= [] 
when_old = 1
for i in 1:length(when_visit) 
    when_new = when_visit[i]
    if i != 1 
        when_old = when_visit[i-1]
    end
    if (when_new - when_old) > interval 
        append!(when_visit_real, when_new) 
    end
end


In [ ]:
which_map = place_map_all_1[:,:,real_cell_index]
test_map = map_field(which_map; threshold = 8/10, bottom_activity= 0)

imshow(test_map)

In [ ]:
py"""
import numpy as np
import itertools

def find_consecutive(x):
    return list((g[0][0], len(g)) 
    for key, group in itertools.groupby(enumerate(x), key=lambda v: v[1]) 
    if key 
    for g in (list(group),))
"""

In [ ]:
whether_in_zone = [(test_map[x_digital[i], y_digital[i]] .== 1) for i in 1:length(x_digital)];
time_duration_outzone = py"find_consecutive"(.!whether_in_zone)
out_zone_time = [time_duration[1]+1 for time_duration in time_duration_outzone]
out_zone_duration = [time_duration[2] for time_duration in time_duration_outzone];
for out_zone_short in findall(out_zone_duration .<125*6)
    whether_in_zone[out_zone_time[out_zone_short]:out_zone_time[out_zone_short]+out_zone_duration[out_zone_short]-1] .= 1
end

In [ ]:
time_duration_zone = py"find_consecutive"(whether_in_zone)
in_zone_time = [time_duration[1]+1 for time_duration in time_duration_zone]
in_zone_duration = [time_duration[2] for time_duration in time_duration_zone];

in_zone_time_valid = in_zone_time[in_zone_duration.>=125]
in_zone_duration_valid = in_zone_duration[in_zone_duration.>=125];

heading_0 = (abs.(heading[in_zone_time_valid].-pi).>=3*pi/4)
heading_180 = (abs.(heading[in_zone_time_valid].-pi).<pi/4);

in_zone_time_downsample = Int32.(round.(in_zone_time_valid./125))
in_zone_duration_downsample = Int32.(round.(in_zone_duration_valid./125));

heading_index = fill(NaN32,length(x_fish_sweep_mean))
for i in 1:length(in_zone_time_downsample)
    start_time = in_zone_time_downsample[i]
    duration = in_zone_duration_downsample[i]
    end_time = start_time+duration-1
    if heading_0[i]
        heading_index[start_time-30:minimum([end_time+30,n_sweeps])].=0
    elseif heading_180[i]
        heading_index[start_time-30:minimum([end_time+30,n_sweeps])].=1
    end
end

In [ ]:
hist(heading[in_zone_time_valid][heading_0])
xlim(0,2*pi)

In [ ]:
hist(heading[in_zone_time_valid][heading_180])
xlim(0,2*pi)

In [ ]:
plot(heading)

In [ ]:
sum(heading_0)

In [ ]:
sum(heading_180)

In [ ]:
x_fish_mm_fit = fit(TrendFilter, x_fish_sweep_mean .* 0.02, 5, 2);
y_fish_mm_fit = fit(TrendFilter, y_fish_sweep_mean .* 0.02, 5, 2);
speed_mm_s = (diff(x_fish_mm_fit.β).^2 .+ diff(y_fish_mm_fit.β).^2).^0.5 ./ 0.5;
speed_mm_s = vcat(speed_mm_s[1], speed_mm_s);
# Moving indices
speed_thr = 0.1
valid_moving_indices = (speed_mm_s .> speed_thr);

In [ ]:
start_index = 1
end_index = 10800

plot_index = trues(n_sweeps)
if end_index<n_sweeps
    plot_index[end_index+1:end] .= false
end
plot_index[1:start_index-1].=false
plot_index[.!valid_moving_indices].=false;

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/figure1/place_cell_examples"
mkpath(fig_path)

In [ ]:
in_zone_time_downsample[heading_0]

In [ ]:
in_zone_time_downsample[heading_180]

In [ ]:
neural_activity = A_dFF_1[:, real_cell_index]
neural_activity = (neural_activity.-numpy.nanmean(neural_activity))./numpy.nanstd(neural_activity);
x_fish_valid = x_fish_sweep_mean
y_fish_valid = y_fish_sweep_mean
heading_index_valid = heading_index




for i in findall(heading_180)
    fig = figure()
    start_time = in_zone_time_downsample[i]
    duration = in_zone_duration_downsample[i]
    end_time = start_time+duration-1
    scatter(x_fish_valid[start_time-30:end_time+30], y_fish_valid[start_time-30:end_time+30], c=1:duration+60)
    
    imshow(img_bg_end_1', cmap="binary", origin="lower")

    axis("off")
end


In [ ]:
neural_activity = A_dFF_1[:, real_cell_index]
neural_activity = (neural_activity.-numpy.nanmean(neural_activity))./numpy.nanstd(neural_activity);
x_fish_valid = x_fish_sweep_mean
y_fish_valid = y_fish_sweep_mean
heading_index_valid = heading_index


fig = figure(dpi=250)
plot(x_fish_valid[heading_index_valid.==0], y_fish_valid[heading_index_valid.==0], "k.", markersize = 0.1)

layer_rank = sortperm(neural_activity[1:end-1])
for i = layer_rank
    if heading_index_valid[i]==0
        cur_value = neural_activity[i]; 
        cur_value < 0 ? cur_value = 0 : (cur_value > 1 ? cur_value = 1 : 1+1)
        if isfinite(cur_value)
            plot(x_fish_valid[i:i+1], y_fish_valid[i:i+1], color = [cur_value, 0, 0], alpha = 0.5, linewidth = 3)
        else
            plot(x_fish_valid[i:i+1], y_fish_valid[i:i+1], color = [0, 0, 0], alpha = 0.1, linewidth = 1.5)
        end
    end
end

imshow(img_bg_end_1', cmap="binary", origin="lower", alpha=0)
xlim(400,2850)
ylim(2000, 780)
axis("off")
fig.savefig(joinpath(fig_path, "$real_cell_index right_traversals.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)

In [ ]:
neural_activity = A_dFF_1[:, real_cell_index]
neural_activity = (neural_activity.-numpy.nanmean(neural_activity))./numpy.nanstd(neural_activity);
x_fish_valid = x_fish_sweep_mean
y_fish_valid = y_fish_sweep_mean
heading_index_valid = heading_index


fig = figure(dpi=250)
plot(x_fish_valid[heading_index_valid.==1], y_fish_valid[heading_index_valid.==1], "k.", markersize = 0.1)

layer_rank = sortperm(neural_activity[1:end-1])
for i = layer_rank
    if heading_index_valid[i]==1
        cur_value = neural_activity[i]; 
        cur_value < 0 ? cur_value = 0 : (cur_value > 1 ? cur_value = 1 : 1+1)
        if isfinite(cur_value)
            plot(x_fish_valid[i:i+1], y_fish_valid[i:i+1], color = [cur_value, 0, 0], alpha = 0.5, linewidth = 3)
        else
            plot(x_fish_valid[i:i+1], y_fish_valid[i:i+1], color = [0, 0, 0], alpha = 0.1, linewidth = 1.5)
        end
    end
end

imshow(img_bg_end_1', cmap="binary", origin="lower", alpha=0)
xlim(400,2850)
ylim(2000, 780)
axis("off")
fig.savefig(joinpath(fig_path, "$real_cell_index left_traversals.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)

In [ ]:
neural_activity = A_dFF_1[:, real_cell_index]
neural_activity = (neural_activity.-numpy.nanmean(neural_activity))./numpy.nanstd(neural_activity);
x_fish_valid = x_fish_sweep_mean
y_fish_valid = y_fish_sweep_mean
heading_index_valid = heading_index
fig = figure(dpi=250)
# plot(x_fish_valid, y_fish_valid, "k.", markersize = 0.1)
for i = 1:length(neural_activity)-1
    if heading_index_valid[i]==1
        cur_value = neural_activity[i]; 
        cur_value < 0 ? cur_value = 0 : (cur_value > 1 ? cur_value = 1 : 1+1)
        if isfinite(cur_value) && plot_index[i]
            plot(x_fish_valid[i:i+1], y_fish_valid[i:i+1], color = [cur_value, 0, 0], alpha = 0.5)
        else
            plot(x_fish_valid[i:i+1], y_fish_valid[i:i+1], color = [0, 0, 0], alpha = 0.1)
        end
    end
end

imshow(img_bg_end_1', cmap="binary", origin="lower")
xlim(150,3150)
ylim(500,2250)
axis("equal")
axis("off")


In [ ]:
neural_activity = A_dFF_1[:, real_cell_index]
neural_activity = (neural_activity.-numpy.nanmean(neural_activity))./numpy.nanstd(neural_activity);
x_fish_valid = x_fish_sweep_mean
y_fish_valid = y_fish_sweep_mean
heading_index_valid = heading_index
fig = figure(dpi=250)
# plot(x_fish_valid, y_fish_valid, "k.", markersize = 0.1)
for i = 1:length(neural_activity)-1
    if heading_index_valid[i]==1
        cur_value = neural_activity[i]; 
        cur_value < 0 ? cur_value = 0 : (cur_value > 1 ? cur_value = 1 : 1+1)
        if isfinite(cur_value) && plot_index[i]
            plot(x_fish_valid[i:i+1], y_fish_valid[i:i+1], color = [cur_value, 0, 0], alpha = 0.5)
        else
            plot(x_fish_valid[i:i+1], y_fish_valid[i:i+1], color = [0, 0, 0], alpha = 0.1)
        end
    end

end
imshow(img_bg_end_1', cmap="binary", origin="lower")
xlim(150,3150)
ylim(2250, 500)
axis("equal")

In [ ]:
@pyimport pandas as pd

In [ ]:
function fill_missing(vec, desired_length)
    [vec; fill(NaN, desired_length-length(vec))]
end

In [ ]:
df_length = sum([heading_index.==0][1])

In [ ]:
d = Dict(
    "dF, rightward traversal" => fill_missing(neural_activity[heading_index.==0], df_length),
    "fish anchor x, rightward traversal" => fill_missing(x_fish_valid[heading_index.==0], df_length),
    "fish anchor y, rightward traversal" => fill_missing(y_fish_valid[heading_index.==0], df_length),
    
    "dF, leftward traversal" => fill_missing(neural_activity[heading_index.==1], df_length),
    "fish anchor x, leftward traversal" => fill_missing(x_fish_valid[heading_index.==1], df_length),
    "fish anchor y, leftward traversal" => fill_missing(y_fish_valid[heading_index.==1], df_length)

)

In [ ]:
df = pd.DataFrame.from_dict(d)
df

In [ ]:
@pyimport openpyxl
path = "../../../figures/source_data/Source Data Fig. 1.xlsx"
writer = pd.ExcelWriter(path, engine = "openpyxl", mode="a")
df.to_excel(writer, sheet_name = "Fig. 1e")
writer.close()